# Final Project
## The Social Informatics of Large Language Models

#### Claire Jordan, Sharon Bures, Elena Solar

In [ ]:
import pandas as pd
import openai

openai.api_base="http://91.107.239.71:80" #"http://127.0.0.1:8000"
openai.api_key="" # enter you API key here

from tqdm import tqdm 
import time 

import json 


In [ ]:

prompt_techniques = ["default", "blurry", "emotional", "chain_of_thought", "persona"]
topics = ["matematical_logical", "coding", "moral_jailbreak"]
positioning = ["beginning", "middle", "end"]

runs = 3



# one dictionary per topic, storing the three positioning-prompts per technique

math_dict = {'default' : ["default_prompt_beginning", "default_prompt_middle", "default_prompt_end"],
             'blurry' : ["blurry_prompt_beginning", "blurry_prompt_middle", "blurry_prompt_end"],
             'emotional' : ["emotional_prompt_beginning", "emotional_prompt_middle", "emotional_prompt_end"],
             'chain_of_thought' : ["cot_prompt_beginning", "cot_prompt_middle", "cot_prompt_end"],
             'persona' : ["persona_prompt_beginning", "persona_prompt_middle", "persona_prompt_end"],
}


coding_dict = {'default' : ["default_prompt_beginning", "default_prompt_middle", "default_prompt_end"],
             'blurry' : ["blurry_prompt_beginning", "blurry_prompt_middle", "blurry_prompt_end"],
             'emotional' : ["emotional_prompt_beginning", "emotional_prompt_middle", "emotional_prompt_end"],
             'chain_of_thought' : ["cot_prompt_beginning", "cot_prompt_middle", "cot_prompt_end"],
             'persona' : ["persona_prompt_beginning", "persona_prompt_middle", "persona_prompt_end"],
}

moral_dict = {'default' : ["default_prompt_beginning", "default_prompt_middle", "default_prompt_end"],
             'blurry' : ["blurry_prompt_beginning", "blurry_prompt_middle", "blurry_prompt_end"],
             'emotional' : ["emotional_prompt_beginning", "emotional_prompt_middle", "emotional_prompt_end"],
             'chain_of_thought' : ["cot_prompt_beginning", "cot_prompt_middle", "cot_prompt_end"],
             'persona' : ["persona_prompt_beginning", "persona_prompt_middle", "persona_prompt_end"],
}

# make one data frame from each dictionary
math_prompts = pd.DataFrame(math_dict, columns = positioning)
coding_prompts = pd.DataFrame(coding_dict, columns = positioning)
moral_prompts = pd.DataFrame(moral_dict, columns = positioning)


In [ ]:
# Function to make prompt
## probably not needed


# def make_prompt(topic, prompt_technique, position):

#     """
#     Function to construct the prompt for the given topic, technique and positioning.

#     Parameters:
#     -----------
#         topic: the topic which shall be asked (mathematical, coding or moral)
#         prompt_technique: the prompt technique, that shall be used (default, blurry, emotional, chain of thought or persona)
#         positioning: positioning of the important sentence (beginning, middle or end)

#     Returns:
#     --------
#         prompt: the final prompt
#     """


#     if topic == "mathematical":
#         prompt = math_prompts[prompt_technique, position]
#         return prompt

#     elif topic == "coding":
#         prompt = coding_prompts[prompt_technique, position]
#         return prompt

#     elif topic == "moral":
#         prompt = moral_prompts[prompt_technique, position]
#         return prompt

#     else:
#         # error, no valid topic given

In [ ]:
# Function to call GPT (both versions)

def call_GPT(topics, prompt_techniques, positions, gpt_model, runs):
    """ 
    Function to call ChatGPT for classifying questions and scenarios as moral issues or not.

    Parameters:
    -----------
        topic: the topic which shall be asked (mathematical, coding or moral)
        prompt_technique: the prompt technique, that shall be used (default, blurry, emotional, chain of thought or persona)
        positioning: positioning of the important sentence (beginning, middle or end)
        gpt_model: the version of gpt to use (gpt-3.5-turbo or gpt-4-1106-preview)

    
    Returns:
    -------
        result: Dictionary conatining all information (on inputs) an results
    """

    # initialize result dictionary
    result = {'topic' : [],
          'prompt_technique' : [],
          'position' : [],
          'run' : [],
          'gpt_version' : [],
          'answer' : [],
          'tokens' : []}
    
    # process bar
    total_iterations = len(topics) * len(prompt_techniques) * len(positions) * len(gpt_model) * len(runs)
    progress_bar = tqdm(total=total_iterations, desc='Iterations')
    iteration_counter = 0

    for topic in topics: 
        for prompt_technique in prompt_techniques:
            for position in positions: 
                
                    
                # make prompt 
                if topic == 'mathematics':
                    prompt = math_prompts[prompt_technique, position]
                
                elif topic == 'coding':
                    prompt = coding_prompts[prompt_technique, position]

                elif topic == 'moral':
                    prompt = moral_prompts[prompt_technique, position]
                # 3 runs
                for i in range(runs):

                    try:
                        resp = openai.ChatCompletion.create(model=gpt_model,
                                            messages=[{"role": "user", "content": prompt}],
                                            n=1)
                        
                        answer = resp['choices'][0]['message']['content']

                        if gpt_model == 'gpt-3.5-turbo':
                            tokens = resp['usage']['completion_tokens'] + resp['usage']['prompt_tokens']
                        
                        elif gpt_model == 'gpt-4-1106-preview':
                            tokens = resp['usage']['completion_tokens'] * 15 + resp['usage']['prompt_tokens'] * 10

                        # Append to result dictionary
                        result['topic'].append(topic)
                        result['prompt_technique'].append(prompt_technique)
                        result['position'].append(position)
                        result['run'].append(i + 1)
                        result['gpt_version'].append(gpt_model)
                        result['answer'].append(answer)
                        result['tokens'].append(tokens)
                            
                        
  

                    except Exception as e:
                        resp = e

                       # Append to result dictionary
                        result['topic'].append(topic)
                        result['prompt_technique'].append(prompt_technique)
                        result['position'].append(position)
                        result['run'].append(i + 1)
                        result['gpt_version'].append(gpt_model)
                        result['answer'].append(f'Error: {e}')
                        result['tokens'].append(tokens)
                        
                    
                    iteration_counter += 1
                    if iteration_counter % 10 == 0:  # Update progress bar every 10 iterations
                        progress_bar.update(10)
    
    progress_bar.close()
    return result
